<a href="https://colab.research.google.com/github/sankalp294/Combat-Solutions-tasks/blob/main/3_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q langchain-core langchain-groq langgraph

import os
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Dict
from langchain_core.messages import HumanMessage, AIMessage

os.environ["GROQ_API_KEY"] = " xyz "

class ChatState(TypedDict):
    user_input: str
    conversation_history: List[Dict[str, str]]
    llm_response: str
    output: str

# NODE 1: User Input
def user_input_node(state: ChatState) -> ChatState:
    """Process user input and prepare for LLM"""
    print(f"\n[USER INPUT NODE] Received: {state['user_input']}")
    return state

# NODE 2: Memory
def memory_node(state: ChatState) -> ChatState:
    """Build context from conversation history"""
    print(f"[MEMORY NODE] Loading conversation history...")

    if state['conversation_history']:
        print(f"   - Found {len(state['conversation_history'])} previous messages")
    else:
        print("   - No previous messages (first interaction)")

    return state

# NODE 3: LLM
def llm_node(state: ChatState) -> ChatState:
    """Call LLM with context and generate response"""
    print(f"[LLM NODE] Generating response...")

    llm = ChatGroq(
        model="llama-3.3-70b-versatile",
        temperature=0.7
    )

    messages = []
    if state['conversation_history']:
        for msg in state['conversation_history']:
            if msg['role'] == "User":
                messages.append(HumanMessage(content=msg['content']))
            elif msg['role'] == "Assistant":
                messages.append(AIMessage(content=msg['content']))

    messages.append(HumanMessage(content=state['user_input']))

    response_message = llm.invoke(messages)
    state['llm_response'] = response_message.content

    state['conversation_history'].append({
        "role": "User",
        "content": state['user_input']
    })
    state['conversation_history'].append({
        "role": "Assistant",
        "content": response_message.content
    })

    return state

# NODE 4: Output
def output_node(state: ChatState) -> ChatState:
    """Format and return final output"""
    print(f"[OUTPUT NODE] Preparing response...")
    state['output'] = state['llm_response']
    return state

# GRAPH
workflow = StateGraph(ChatState)


workflow.add_node("user_input", user_input_node)
workflow.add_node("memory", memory_node)
workflow.add_node("llm", llm_node)
workflow.add_node("output", output_node)

workflow.set_entry_point("user_input")
workflow.add_edge("user_input", "memory")
workflow.add_edge("memory", "llm")
workflow.add_edge("llm", "output")
workflow.add_edge("output", END)

chatbot = workflow.compile()

print("=" * 70)
print("TESTING CHATBOT WITH MEMORY")
print("=" * 70)

conversation_history = []

# Message 1
print("\n" + "="*70)
print("MESSAGE 1")
print("="*70)
result = chatbot.invoke({
    "user_input": "Hi! My name is Sankalp and I'm a software engineer.",
    "conversation_history": conversation_history,
    "llm_response": "",
    "output": ""
})
conversation_history = result['conversation_history']
print(f"\nBot: {result['output']}")

# Message 2
print("\n" + "="*70)
print("MESSAGE 2")
print("="*70)
result = chatbot.invoke({
    "user_input": "What's my name?",
    "conversation_history": conversation_history,
    "llm_response": "",
    "output": ""
})
conversation_history = result['conversation_history']
print(f"\nBot: {result['output']}")

# Message 3
print("\n" + "="*70)
print("MESSAGE 3")
print("="*70)
result = chatbot.invoke({
    "user_input": "What's my profession?",
    "conversation_history": conversation_history,
    "llm_response": "",
    "output": ""
})
conversation_history = result['conversation_history']
print(f"\nBot: {result['output']}")

# Message 4
print("\n" + "="*70)
print("MESSAGE 4")
print("="*70)
result = chatbot.invoke({
    "user_input": "Can you summarize what you know about me?",
    "conversation_history": conversation_history,
    "llm_response": "",
    "output": ""
})
conversation_history = result['conversation_history']
print(f"\nBot: {result['output']}")

# Message 5
print("\n" + "="*70)
print("MESSAGE 5")
print("="*70)
result = chatbot.invoke({
    "user_input": "Tell me a programming joke!",
    "conversation_history": conversation_history,
    "llm_response": "",
    "output": ""
})
conversation_history = result['conversation_history']
print(f"\nBot: {result['output']}")


print("\n" + "="*70)
print("COMPLETE CONVERSATION HISTORY")
print("="*70)
for i, msg in enumerate(conversation_history, 1):
    print(f"{i}. {msg['role']}: {msg['content']}")

print("\n" + "="*70)
print("CHATBOT TEST COMPLETE - Memory is working!")
print("="*70)

TESTING CHATBOT WITH MEMORY

MESSAGE 1

[USER INPUT NODE] Received: Hi! My name is Sankalp and I'm a software engineer.
[MEMORY NODE] Loading conversation history...
   - No previous messages (first interaction)
[LLM NODE] Generating response...
[OUTPUT NODE] Preparing response...

Bot: Nice to meet you, Sankalp! It's great to connect with a software engineer. What kind of projects have you been working on lately, or what areas of software development interest you the most?

MESSAGE 2

[USER INPUT NODE] Received: What's my name?
[MEMORY NODE] Loading conversation history...
   - Found 2 previous messages
[LLM NODE] Generating response...
[OUTPUT NODE] Preparing response...

Bot: Your name is Sankalp.

MESSAGE 3

[USER INPUT NODE] Received: What's my profession?
[MEMORY NODE] Loading conversation history...
   - Found 4 previous messages
[LLM NODE] Generating response...
[OUTPUT NODE] Preparing response...

Bot: You're a software engineer.

MESSAGE 4

[USER INPUT NODE] Received: Can you